### 인공지능융합특론 0517 과제
- 주제 : 회식자리 친밀도 측정
  회식에 참석한 인원들을 두 그룹으로 나누어 서로 마주 앉게 배치
  서로 마주 앉는 사람들 간의 친밀도 합이 최대가 되도록 그룹을 나눈다.
  즉, Max-cut 문제로 모델링하여 두 그룹 간의 간선 가중치(친밀도)를 최대화한다. 
- 참석자: 총 5명 (부장 (B), 과장 (M), 대리 (S), 사원 (E), 인턴 (I))
- 그래프 모델링 
  정점(Vertex): 참석자 각자 (B, M, S, E, I)
  간선(Edge): 두 참석자 간의 친밀도 관계
  가중치(Weight): 친밀도 점수
- 집합 V를 A와 B로 나눔 (A ∪ B = V, A ∩ B = ∅) : A와 B 사이에 있는 간선들의 가중치 합을 최대화

*-* Phase-3. max_cut_algorithm.ipynb 파일을 기반하여 Qiskit 을 이용한 소스작성

In [ ]:
# pip 확인 및 qiskit 버전 확인
!pip --version
!pip list | grep qiskit

In [ ]:
!pip uninstall -y qiskit

In [ ]:
!pip install "qiskit==0.46.0"

In [ ]:
# 셀 1: 필수 패키지 설치 (Colab 사용 시 실행) -> 버전 변경 
!pip install "qiskit==0.46.0"
!pip install "qiskit-aer==0.12.0"
!pip install "qiskit-optimization==0.5.0"
!pip install "qiskit-algorithms==0.2.0"
!pip install "qiskit-ibm-provider==0.5.0"
!pip install "qiskit-ibm-runtime==0.13.0"

### 세팅

In [25]:
# 셀 2: 필요한 모듈 불러오기
from qiskit import Aer
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.applications import Maxcut
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_provider import least_busy

import networkx as nx
import matplotlib.pyplot as plt


### IBM 연동 

In [28]:
"""
# ibm runtime 을 이용한 방법 <실패>
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options, Session

QiskitRuntimeService.save_account(
    channel="ibm_quantum", 
    token="", 
    overwrite=True)
service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.least_busy(min_num_qubits=5)
print(f"Selected backend: {backend}")

# 옵션 설정
options = Options()
options.optimization_level = 3  # 최적화 레벨 설정
options.resilience_level = 1    # 오류 완화 레벨 설정

# Sampler 설정
session = Session(backend=backend)
sampler = Sampler(session=session, options=options)
"""

# IBM Quantum 계정 로드
provider = IBMProvider(
    token="<ibm-quantum-token>" 
    )

# 사용 가능한 백엔드 중 가장 바쁘지 않은 백엔드 선택
backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 5 and not x.configuration().simulator))
print(f"Selected backend: {backend}")
quantum_instance = QuantumInstance(backend=backend)


Selected backend: <IBMBackend('ibm_sherbrooke')>


/var/folders/3d/dxbhq4214vqddsxgwxm891gm0000gn/T/ipykernel_7118/649297999.py:31: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance = QuantumInstance(backend=backend)


In [ ]:
# 셀 3: 문제 정의 - 참석자와 친밀도
nodes = ["부장", "과장", "대리", "사원", "인턴"]
node_index = {name: i for i, name in enumerate(nodes)}
edges = [
    ("부장", "과장", 8),
    ("부장", "대리", 3),
    ("부장", "사원", 1),
    ("부장", "인턴", 1),
    ("과장", "대리", 7),
    ("과장", "사원", 2),
    ("대리", "사원", 6),
    ("대리", "인턴", 4),
    ("사원", "인턴", 5)
]

# 📌 NetworkX 그래프 생성 (정수 노드 사용)
G = nx.Graph()
G.add_nodes_from(range(len(nodes)))  # 정수 노드
for u, v, w in edges:
    G.add_edge(node_index[u], node_index[v], weight=w)

# ✅ MaxCut 문제로 변환
maxcut = Maxcut(G)
problem = maxcut.to_quadratic_program()
print(problem.prettyprint())

# 셀 5: QAOA 설정
#algorithm_globals.random_seed = 42
#backend = Aer.get_backend("aer_simulator_statevector")
#quantum_instance = QuantumInstance(backend=backend)

# QAOA 초기화 및 instance 설정
qaoa = QAOA(optimizer=COBYLA(), reps=1, quantum_instance=quantum_instance)
#qaoa = QAOA(optimizer=COBYLA(), reps=1, sampler=sampler, initial_point=[1.0, 1.0])

##### !!! 연산자 변환 작업 QAOA가 반환하는 SparsePauliOp → PauliSumOp
from qiskit_optimization.converters import QuadraticProgramToQubo
qubo_converter = QuadraticProgramToQubo()
qubo_problem = qubo_converter.convert(problem)
operator, offset = qubo_problem.to_ising()

# 변환
if not isinstance(operator, PauliSumOp):
    #operator = PauliSumOp.from_operator(operator)
    operator = PauliSumOp(operator)

# QAOA 실행 후 커스텀 MinimumEigenOptimizer 사용
from qiskit.algorithms import MinimumEigensolverResult

class PatchedQAOA(QAOA):
    def compute_minimum_eigenvalue(self, operator, aux_operators=None):
        if not isinstance(operator, PauliSumOp):
            operator = PauliSumOp(operator)
        return super().compute_minimum_eigenvalue(operator, aux_operators)

patched_qaoa = PatchedQAOA(optimizer=COBYLA(), reps=1, quantum_instance=quantum_instance)
#patched_qaoa = PatchedQAOA(optimizer=COBYLA(), reps=1, sampler=sampler)
optimizer = MinimumEigenOptimizer(patched_qaoa)

# 셀 6: 문제 해결
result = optimizer.solve(problem)
solution = maxcut.interpret(result)

print("최적 해:", result)
print("MaxCut 결과 (0과 1로 그룹 분할):", solution)

""" 결과 5/21
FAILURE: Can not get job id, Resubmit the qobj to get job id. Terra job error: 'Error submitting job: \'404 Client Error: Not Found for url: https://api.quantum.ibm.com/runtime/jobs. {"errors":[{"code":1211,"message":"Program not found.","solution":"Make sure you use a valid program name, such as \\\'sampler\\\' or \\\'estimator\\\'. Qiskit Runtime no longer supports the \\\'backend.run\\\' interface. Refer to the migration guide (https://docs.quantum.ibm.com/migration-guides/qiskit-runtime) for instructions to migrate to the primitives.","more_info":"https://docs.quantum-computing.ibm.com/errors"}]}\''
qiskit 버전이 너무 오래되어 최신버전으로 접근해야 하는 이슈 발생 
"""